# Pipeline: Coach Awards

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.window import Window
from pyspark.sql.functions import col, desc, when, dense_rank

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("CoachesApp")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

### Prepare Teams Table

In [2]:
# Load the teams table
teams = spark.read.format("csv").option("header", "true").load("gs://data_de2023_qjsol/Teams.csv") #  use your gcp bucket name. 

# Only keep columns of interest
teams = teams.select("year", "lgID", "tmID", "name", "Pts", "W", "SoW")
teams.na.drop(subset = ["year", "lgID", "tmID", "name", "Pts", "W"]) # we allow NULL values in SoW as these indicate SoW = 0

# Replace NULL with 0 in SoW
teams = teams.fillna("0", "SoW")

In [3]:
# There is a spelling error in the team names.
# We noticed that 'Chicago Blackhawks' and 'Chicago Black Hawks' are mapped to the same team ID.
teams.createOrReplaceTempView("teams")
teams.show()

# SQL query to display that 'Chicago Blackhawks' and 'Chicago Black Hawks' are mapped to the same team ID.
team_names = spark.sql("""
    SELECT name, tmID, COUNT(*) FROM teams
    WHERE name = 'Chicago Blackhawks' OR name = 'Chicago Black Hawks'
    GROUP BY name, tmID
""")
team_names.show()

# Resolve spelling error in team name
teams = teams.withColumn('name', when(col('name') == 'Chicago Blackhawks', 'Chicago Black Hawks').otherwise(col('name')))

+----+----+----+--------------------+---+---+---+
|year|lgID|tmID|                name|Pts|  W|SoW|
+----+----+----+--------------------+---+---+---+
|1909| NHA| COB| Cobalt Silver Kings|  8|  4|  0|
|1909| NHA| HAI|Haileybury Hockey...|  8|  4|  0|
|1909| NHA| LES|       Les Canadiens|  4|  2|  0|
|1909| NHA| MOS|  Montreal Shamrocks|  7|  3|  0|
|1909| NHA| MOW|  Montreal Wanderers| 22| 11|  0|
|1909| NHA| OT1|     Ottawa Senators| 18|  9|  0|
|1909| NHA| REN|Renfrew Creamery ...| 17|  8|  0|
|1910| NHA| MOC|  Montreal Canadiens| 16|  8|  0|
|1910| NHA| MOW|  Montreal Wanderers| 14|  7|  0|
|1910| NHA| OT1|     Ottawa Senators| 26| 13|  0|
|1910| NHA| QU1|     Quebec Bulldogs|  8|  4|  0|
|1910| NHA| REN|Renfrew Creamery ...| 16|  8|  0|
|1911| NHA| MOC|  Montreal Canadiens| 16|  8|  0|
|1911| NHA| MOW|  Montreal Wanderers| 18|  9|  0|
|1911| NHA| OT1|     Ottawa Senators| 18|  9|  0|
|1911| NHA| QU1|     Quebec Bulldogs| 20| 10|  0|
|1911|PCHA| NWR|New Westminster R...| 18|  9|  0|


In [4]:
# Show schema
teams.printSchema()

# Convert year, points, wins and  rank from string to integer type
teams = teams.withColumn("year", col("year").cast("int")) \
             .withColumn("Pts", col("Pts").cast("int")) \
             .withColumn("W", col("W").cast("int")) \
             .withColumn("SoW", col("SoW").cast("int"))

# Show new schema and top 5 rows
teams.printSchema()
teams.show(5)

root
 |-- year: string (nullable = true)
 |-- lgID: string (nullable = true)
 |-- tmID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Pts: string (nullable = true)
 |-- W: string (nullable = true)
 |-- SoW: string (nullable = false)

root
 |-- year: integer (nullable = true)
 |-- lgID: string (nullable = true)
 |-- tmID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Pts: integer (nullable = true)
 |-- W: integer (nullable = true)
 |-- SoW: integer (nullable = true)

+----+----+----+--------------------+---+---+---+
|year|lgID|tmID|                name|Pts|  W|SoW|
+----+----+----+--------------------+---+---+---+
|1909| NHA| COB| Cobalt Silver Kings|  8|  4|  0|
|1909| NHA| HAI|Haileybury Hockey...|  8|  4|  0|
|1909| NHA| LES|       Les Canadiens|  4|  2|  0|
|1909| NHA| MOS|  Montreal Shamrocks|  7|  3|  0|
|1909| NHA| MOW|  Montreal Wanderers| 22| 11|  0|
+----+----+----+--------------------+---+---+---+
only showing top 5 rows



### Prepare Coaches Table

In [5]:
# Load the coaches table
coaches = spark.read.format("csv").option("header", "true").load("gs://data_de2023_qjsol/Coaches.csv") #  use your gcp bucket name. 

# Only keep columns of interest
coaches = coaches.select("coachID", "year", "tmID")
coaches.na.drop("any")

# Show schema
coaches.printSchema()

# Convert year from string to integer type
coaches = coaches.withColumn("year", col("year").cast("int"))

# Show new schema and top 5 rows
coaches.printSchema()
coaches.show(5)

root
 |-- coachID: string (nullable = true)
 |-- year: string (nullable = true)
 |-- tmID: string (nullable = true)

root
 |-- coachID: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- tmID: string (nullable = true)

+---------+----+----+
|  coachID|year|tmID|
+---------+----+----+
|abelsi01c|1952| CHI|
|abelsi01c|1953| CHI|
|abelsi01c|1957| DET|
|abelsi01c|1958| DET|
|abelsi01c|1959| DET|
+---------+----+----+
only showing top 5 rows



### Prepare AwardsCoaches Table

In [6]:
# Load the awards coaches table
awards = spark.read.format("csv").option("header", "true").load("gs://data_de2023_qjsol/AwardsCoaches.csv") #  use your gcp bucket name. 

# Only keep columns of interest
awards = awards.select("coachID", "award", "year")
awards.na.drop("any")

# Show schema
awards.printSchema()

# Rename columns for clarity
awards = awards.withColumnRenamed("year", "award_year")
awards = awards.withColumnRenamed("coachID", "award_coachID")

# Convert award_year from string to integer type
awards = awards.withColumn("award_year", col("award_year").cast("int"))

# Show new schema and top 5 rows
awards.printSchema()
awards.show(5)

root
 |-- coachID: string (nullable = true)
 |-- award: string (nullable = true)
 |-- year: string (nullable = true)

root
 |-- award_coachID: string (nullable = true)
 |-- award: string (nullable = true)
 |-- award_year: integer (nullable = true)

+-------------+--------------------+----------+
|award_coachID|               award|award_year|
+-------------+--------------------+----------+
|   patrile01c| First Team All-Star|      1930|
|   irvindi01c|Second Team All-Star|      1930|
|   patrile01c| First Team All-Star|      1931|
|   irvindi01c|Second Team All-Star|      1931|
|   patrile01c| First Team All-Star|      1932|
+-------------+--------------------+----------+
only showing top 5 rows



### Select NHL team statistics from 1931 onwards

In [7]:
# Select teams of National Hockey League (NHL)
nhl_teams = teams.filter(teams['lgID'] == 'NHL').drop('lgID')
nhl_teams.show(5)

# Only keep teams with year => 1931 (because awards were only given from 1931 onwards)
teams_1931 = nhl_teams.filter(nhl_teams['year'] >= 1931)

+----+----+------------------+---+---+---+
|year|tmID|              name|Pts|  W|SoW|
+----+----+------------------+---+---+---+
|1917| MTL|Montreal Canadiens| 26| 13|  0|
|1917| MTW|Montreal Wanderers|  2|  1|  0|
|1917| OTS|   Ottawa Senators| 18|  9|  0|
|1917| TOA|    Toronto Arenas| 26| 13|  0|
|1918| MTL|Montreal Canadiens| 20| 10|  0|
+----+----+------------------+---+---+---+
only showing top 5 rows



### Compute ROW

In [8]:
# Compute ROW
# The ROW subtracts the number of wins a team secures through the shootout from their overall total. 
# It is then used as a tiebreaker between two teams tied in the standings. 
# The team who has a higher ROW, will be placed higher in the standings.
teams_1931 = teams_1931.withColumn("ROW", col("W") - col("SoW")).drop("W").drop("SoW")
teams_1931.show()

+----+----+-------------------+---+---+
|year|tmID|               name|Pts|ROW|
+----+----+-------------------+---+---+
|1931| BOS|      Boston Bruins| 42| 15|
|1931| CHI|Chicago Black Hawks| 47| 18|
|1931| DTF|    Detroit Falcons| 46| 18|
|1931| MTL| Montreal Canadiens| 57| 25|
|1931| MTM|   Montreal Maroons| 45| 19|
|1931| NYA| New York Americans| 40| 16|
|1931| NYR|   New York Rangers| 54| 23|
|1931| TOR|Toronto Maple Leafs| 53| 23|
|1932| BOS|      Boston Bruins| 58| 25|
|1932| CHI|Chicago Black Hawks| 44| 16|
|1932| DET|  Detroit Red Wings| 58| 25|
|1932| MTL| Montreal Canadiens| 41| 18|
|1932| MTM|   Montreal Maroons| 50| 22|
|1932| NYA| New York Americans| 41| 15|
|1932| NYR|   New York Rangers| 54| 23|
|1932| OTS|    Ottawa Senators| 32| 11|
|1932| TOR|Toronto Maple Leafs| 54| 24|
|1933| BOS|      Boston Bruins| 41| 18|
|1933| CHI|Chicago Black Hawks| 51| 20|
|1933| DET|  Detroit Red Wings| 58| 24|
+----+----+-------------------+---+---+
only showing top 20 rows



### Extract top 3 teams per year

In [9]:
# Define a Window specification
window_spec = Window.partitionBy("year").orderBy(col("Pts").desc(), col("ROW").desc())

# Apply dense rank on the windows
ranked_teams = teams_1931.withColumn("dense_rank", dense_rank().over(window_spec))

# Filter rows where dense_rank is less than or equal to 3
top_three_teams = ranked_teams.filter(col("dense_rank") <= 3)

# Show the resulting DataFrame
top_three_teams.show()

+----+----+-------------------+---+---+----------+
|year|tmID|               name|Pts|ROW|dense_rank|
+----+----+-------------------+---+---+----------+
|1931| MTL| Montreal Canadiens| 57| 25|         1|
|1931| NYR|   New York Rangers| 54| 23|         2|
|1931| TOR|Toronto Maple Leafs| 53| 23|         3|
|1932| BOS|      Boston Bruins| 58| 25|         1|
|1932| DET|  Detroit Red Wings| 58| 25|         1|
|1932| TOR|Toronto Maple Leafs| 54| 24|         2|
|1932| NYR|   New York Rangers| 54| 23|         3|
|1933| TOR|Toronto Maple Leafs| 61| 26|         1|
|1933| DET|  Detroit Red Wings| 58| 24|         2|
|1933| CHI|Chicago Black Hawks| 51| 20|         3|
|1934| TOR|Toronto Maple Leafs| 64| 30|         1|
|1934| BOS|      Boston Bruins| 58| 26|         2|
|1934| CHI|Chicago Black Hawks| 57| 26|         3|
|1935| DET|  Detroit Red Wings| 56| 24|         1|
|1935| MTM|   Montreal Maroons| 54| 22|         2|
|1935| TOR|Toronto Maple Leafs| 52| 23|         3|
|1936| DET|  Detroit Red Wings|

### Join with Coaches

In [10]:
# Join with Coaches table
intermediate = top_three_teams.join(coaches, ["tmID", "year"], "left")

### Join with AwardsCoaches

In [11]:
# Join with AwardsCoaches table
joinExpression = (intermediate["coachID"] == awards["award_coachID"]) & (intermediate["year"] >= awards["award_year"])

intermediate = intermediate.join(awards, joinExpression, "left").drop("award_coachID").drop("award_year")
intermediate.show()

+----+----+-------------------+---+---+----------+----------+--------------------+
|tmID|year|               name|Pts|ROW|dense_rank|   coachID|               award|
+----+----+-------------------+---+---+----------+----------+--------------------+
| MTL|1931| Montreal Canadiens| 57| 25|         1| hartce01c|                NULL|
| NYR|1931|   New York Rangers| 54| 23|         2|patrile01c| First Team All-Star|
| NYR|1931|   New York Rangers| 54| 23|         2|patrile01c| First Team All-Star|
| TOR|1931|Toronto Maple Leafs| 53| 23|         3|irvindi01c|Second Team All-Star|
| TOR|1931|Toronto Maple Leafs| 53| 23|         3|irvindi01c|Second Team All-Star|
| TOR|1931|Toronto Maple Leafs| 53| 23|         3|duncaar01c|                NULL|
| BOS|1932|      Boston Bruins| 58| 25|         1| rossar01c|                NULL|
| DET|1932|  Detroit Red Wings| 58| 25|         1|adamsja01c|                NULL|
| TOR|1932|Toronto Maple Leafs| 54| 24|         2|irvindi01c|Second Team All-Star|
| TO

### Count awards per category for each team in each year

In [12]:
intermediate.createOrReplaceTempView("intermediate")
intermediate.show()

# Create a DataFrame with all combinations of teams, years, and award categories
all_combinations = spark.sql("""
    SELECT DISTINCT tmID, year, name, Pts, ROW, dense_rank, category
    FROM intermediate
    CROSS JOIN (SELECT DISTINCT award AS category FROM intermediate)
""")
all_combinations.createOrReplaceTempView("all_combinations")
all_combinations.show()

# Left join to include all combinations in the result
result = spark.sql("""
    SELECT ac.tmID, ac.year, ac.name, ac.Pts, ac.ROW, ac.dense_rank, ac.category AS award, COALESCE(COUNT(i.award), 0) AS award_count
    FROM all_combinations ac
    LEFT JOIN intermediate i
    ON ac.tmID = i.tmID AND ac.year = i.year AND ac.category = i.award
    GROUP BY ac.tmID, ac.year, ac.name, ac.Pts, ac.ROW, ac.dense_rank, ac.category
""")
result.show()
result.printSchema()

# Remove rows where award is NULL (these rows occur in 'intermediate' when no awards were received by that team in that year)
result = result.filter(col("award").isNotNull())

+----+----+-------------------+---+---+----------+----------+--------------------+
|tmID|year|               name|Pts|ROW|dense_rank|   coachID|               award|
+----+----+-------------------+---+---+----------+----------+--------------------+
| MTL|1931| Montreal Canadiens| 57| 25|         1| hartce01c|                NULL|
| NYR|1931|   New York Rangers| 54| 23|         2|patrile01c| First Team All-Star|
| NYR|1931|   New York Rangers| 54| 23|         2|patrile01c| First Team All-Star|
| TOR|1931|Toronto Maple Leafs| 53| 23|         3|irvindi01c|Second Team All-Star|
| TOR|1931|Toronto Maple Leafs| 53| 23|         3|irvindi01c|Second Team All-Star|
| TOR|1931|Toronto Maple Leafs| 53| 23|         3|duncaar01c|                NULL|
| BOS|1932|      Boston Bruins| 58| 25|         1| rossar01c|                NULL|
| DET|1932|  Detroit Red Wings| 58| 25|         1|adamsja01c|                NULL|
| TOR|1932|Toronto Maple Leafs| 54| 24|         2|irvindi01c|Second Team All-Star|
| TO

## Store the result in BigQuery

In [13]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "data_de2023_qjsol"  # use your bucket 
spark.conf.set('temporaryGcsBucket', bucket)

# Saving the data to BigQuery
result.write.format('bigquery').option('table', 'dataengineering2023-398611.assignment2.coaches').mode("append").save() # use your project-id

In [14]:
# Stop the spark context
spark.stop()